# Setup

In [1]:
import numpy as np
from ortools.linear_solver import pywraplp
solver = pywraplp.Solver.CreateSolver('SCIP')

# Definindo as variáveis

In [10]:
cidades = ['D', 'M', 'G', 'J', 'P']
distancias = np.array(
    [[0,467,340,327,358],
     [467,0,497,677,843],
     [340,497,0,454,762],
     [327,677,454,0,441],
     [358,843,762,441,0]]
)
N = distancias.shape[0]
X = {}
for i in range(N):
    for j in range(N):
        X[i,j] = solver.IntVar(0, 1, 'X_[{},{}]'.format(i, j))

# Adicionando restricao de $n - 1$ caminhos

In [3]:
solver.Add(sum([X[i,j] for i in range(N) for j in range(N)]) == N - 1)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7fc36c9d7f00> >

# Adicionando restrições contra ciclos

In [4]:
# definindo a funcao para gerar subconjuntos validos, onde S != {} e S != cidades
from itertools import chain, combinations
def subconjuntos_validos(iterable):
    "subconjuntos_validos([1,2,3]) --> (1,) (2,) (3,) (1,2) (1,3) (2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))

# definindo o conjunto de partes
conjunto_de_partes = subconjuntos_validos(range(len(cidades)))

# iterando em cada subconjunto e adicionando a restrição
# sum(X_ij) <= |S| - 1
for S in conjunto_de_partes:
    solver.Add(sum([X[i,j] for j in S for i in S]) <= len(S) - 1)
            

# Adicionando Função Objetivo

In [5]:
solver.Minimize(solver.Sum([distancias[i][j] * X[i, j] for i in range(N) for j in range(N)]))

# Resolvendo o problema

In [6]:
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    for i in range(N):
        for j in range(N):
            if X[i,j].solution_value() == 1: # o caminho está na AGM
                print('Conexão: {} - {}, custo: {}'.format(cidades[i], cidades[j], distancias[i][j]))
    print('Custo total: {}'.format(solver.Objective().Value()))
else:
    print('Não existe árvore geradora mínima')

Conexão: D - M, custo: 467
Conexão: D - G, custo: 340
Conexão: D - J, custo: 327
Conexão: P - D, custo: 358
Custo total: 1492.0
